In [6]:
# Install sumo binary (without traci) from the official repos
!sudo apt install cmake swig libgtest-dev python-pygame python-scipy autoconf libtool pkg-config libgdal-dev libxerces-c-dev libproj-dev libfox-1.6-dev libxml2-dev libxslt1-dev build-essential curl unzip flex bison python python-dev python3-dev sumo sumo-tools sumo-doc
!sudo pip install cmake cython gym==0.12.0 pyprind nose2 cached_property joblib dill lz4 ray==0.6.1 setproctitle psutil opencv-python boto3 redis

# get flow and switch to proper commit
!cd /content && git clone https://github.com/flow-project/flow.git  && cd flow

# get traci from sumo repo at the right commit
!cd /content && git clone https://github.com/eclipse/sumo.git && cd sumo

# git cil4sys code
!cd /content && git clone https://github.com/cyrilhokage/DQN_CIL4SYS.git

# sometimes necessary, colab/gh cash ?
!cd /content/notebooks/ && git pull

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
python is already the newest version (2.7.15~rc1-1).
python-dev is already the newest version (2.7.15~rc1-1).
unzip is already the newest version (6.0-21ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
curl is already the newest version (7.58.0-2ubuntu3.8).
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.3).
python3-dev is already the newest version (3.6.7-1~18.04).
The following additional packages will be installed:
  automake autotools-dev binfmt-support default-libmysqlclient-dev fastjar
  file fonts-freefont-ttf googletest jarwrapper javascript-common
  libarmadillo-dev libarpack2-dev libbison-dev libblas-dev libblas3
  libcups2-dev libcupsfilters-dev libcupsfilters1 libcupsimage2
  libcupsimage2-dev libdap-dev libdapserver7v5

In [7]:
# check flow version
!cd /content/flow && git status

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


In [8]:
# check traci version
!cd /content/sumo && git status

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


In [9]:
# check sumo version
!sumo --version

SUMO Version 0.32.0
 Build features: x86_64-pc-linux-gnu TRACI PROJ GDAL GUI
 Copyright (C) 2001-2017 German Aerospace Center (DLR) and others; http://sumo.dlr.de

SUMO Version 0.32.0 is part of SUMO.
SUMO is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program. If not, see http://www.gnu.org/licenses/gpl.html


In [10]:
# cannot import ray otherwise, see https://github.com/ray-project/ray/issues/5033
!pip uninstall -y pyarrow

In [0]:
# Restart the kernel and run the cells bellow (no need to run the ones above)

# It looks like in order the filesystem on colab notebooks to update, there is a need to restart the kernel. The filesystem isn't touched however, so no need to run the cells above.

In [0]:
import os
import sys

os.environ['SUMO_HOME'] = "/content/sumo"

if not '/content/flow' in sys.path: # so we can rerun cell safely
  sys.path.append('/content/sumo/tools')
  sys.path.append('/content/flow')
  os.environ['PYTHONPATH'] += ":/content/sumo/tools:/content/flow"


In [0]:
# the TestEnv environment is used to simply simulate the network
from flow.envs import TestEnv

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# all other imports are standard
from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams

from flow.networks import Network

In [0]:
from flow.core.params import InFlows

inflow = InFlows()
inflow.add(veh_type="human",
           edge="4794817",
           vehs_per_hour=100,
           depart_speed=10,
           color="green")
inflow.add(veh_type="human",
            edge="4783299#0",
            vehs_per_hour=100,
            depart_lane="random",
            depart_speed="random",
            color="red")
inflow.add(veh_type="human",
           edge="-100822066",
           probability= 0.1,
           depart_lane= 1,  # left lane
           depart_speed= "max",
           begin= 60,  # 1 minute
           number= 30,
           color= "white")
inflow.add(veh_type="human",
            edge="155558218",
            period=2,
            depart_lane="random",
            depart_speed="random",
            color="white")

In [0]:
# Specifie les noms des edges du network dont les vehicules peuvent être originaire
EDGES_DISTRIBUTION = [
    "-100822066",
    "4794817",
    "4783299#0",
    "155558218",
]

In [0]:
# créer une nouvelle classe Network pour spécifier les itinéraires possibles
class IssyOSMNetwork(Network):

    def specify_routes(self, net_params):
        return {
            "-100822066": [ #N
                "-100822066",
                "-352962858#1",
                "-352962858#0",
                "-4786940#1",
                 "-4786940#0",
            ],
            
            "4794817" : [ #Loop
                "4794817",
                "4786972#0",
                "4786972#1",
                "4786972#2",
                "4786965#1",
                "4786965#2",
                "4786965#3",
                "4795729",
                "-352962858#1",
                "4795742#0",
                "4795742#1",
                "4786965#3",
                "4786965#4",
                "4786965#5",
            ],
            
            "4783299#0": [    #E
                "4783299#0",
                "4783299#1",
                "4783299#2",
                "4783299#3",
                "4783299#4",
                "4783299#5",
                "4783299#6",
                "4786940#0",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],
            
            "155558218": [
                "155558218",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],     
        }

In [0]:
# import the base environment class
from flow.envs import Env
from gym.spaces.box import Box
from gym.spaces import Tuple
import numpy as np

In [0]:
class myEnv(Env): # update my environment class

    @property
    def action_space(self):
        num_actions = self.initial_vehicles.num_rl_vehicles
        accel_ub = self.env_params.additional_params["max_accel"]
        accel_lb = - abs(self.env_params.additional_params["max_decel"])

        return Box(low=accel_lb,
                   high=accel_ub,
                   shape=(num_actions,))
        
    @property
    def observation_space(self):
        return Box(
            low=0,
            high=float("inf"),
            shape=(2*self.initial_vehicles.num_vehicles,))
        
    def _apply_rl_actions(self, rl_actions):
        # the names of all autonomous (RL) vehicles in the network
        rl_ids = self.k.vehicle.get_rl_ids()
        # use the base environment method to convert actions into accelerations for the rl vehicles
        self.k.vehicle.apply_acceleration(rl_ids, rl_actions)

    def get_state(self, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()
        # we use the get_absolute_position method to get the positions of all vehicles
        pos = [self.k.vehicle.get_x_by_id(veh_id) for veh_id in ids]
        # we use the get_speed method to get the velocities of all vehicles
        vel = [self.k.vehicle.get_speed(veh_id) for veh_id in ids]
        # the speeds and positions are concatenated to produce the state
        return np.concatenate((pos, vel))

    def compute_reward(self, rl_actions, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()
        # we next get a list of the speeds of all vehicles in the network
        speeds = self.k.vehicle.get_speed(ids)
        # finally, we return the average of all these speeds as the reward
        return np.mean(speeds)                        


In [0]:
HORIZON = 1000

In [19]:
from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS

ADDITIONAL_ENV_PARAMS = {"max_accel": 1, "max_decel": 1}

# SUMO PARAM
sumo_params = SumoParams(sim_step=0.1, render=False)

# create VEHICLE
vehicles = VehicleParams()
vehicles.add(veh_id="human",
             acceleration_controller=(IDMController, {}),
             num_vehicles=22)

# ENVIRONMENT PARAM
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=HORIZON)

# NETWORK PARAM
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params, inflows=inflow, osm_path='DQN_CIL4SYS/notebooks/networks/issy.osm')

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION) #spacing="random",

# dictionnaire FLOW_PARAM
flow_params = dict( exp_tag  = 'ISSY_RL_test',
                    env_name = myEnv,  # using my new environment for the simulation
                    network  = network,
                    simulator='traci',
                    sim      = sumo_params,
                    env      = env_params,
                    net      = net_params,
                    veh      = vehicles,
                    initial  = initial_config)

# create EXPERIMENT with class created
exp = Experiment(flow_params)

# RUN SIMULATION SUMO
_ = exp.run(1)


Error during start: Traceback (most recent call last):
  File "/content/flow/flow/core/kernel/simulation/traci.py", line 163, in start_simulation
    traci_connection.simulationStep()
  File "/content/sumo/tools/traci/connection.py", line 323, in simulationStep
    result = self._sendExact()
  File "/content/sumo/tools/traci/connection.py", line 101, in _sendExact
    raise FatalTraCIError("connection closed by SUMO")
traci.exceptions.FatalTraCIError: connection closed by SUMO

Error during start: Traceback (most recent call last):
  File "/content/flow/flow/core/kernel/simulation/traci.py", line 163, in start_simulation
    traci_connection.simulationStep()
  File "/content/sumo/tools/traci/connection.py", line 323, in simulationStep
    result = self._sendExact()
  File "/content/sumo/tools/traci/connection.py", line 101, in _sendExact
    raise FatalTraCIError("connection closed by SUMO")
traci.exceptions.FatalTraCIError: connection closed by SUMO

Error during start: Traceback (mos

KeyboardInterrupt: ignored

In [0]:
EDGES_DISTRIBUTION